# [BOJ14558] Too Many Traps

## 문제

혜아는 $N$개의 신발을 신고 있다. $i$번째 신발의 능력치를 $A_i$라고 하자. 혜아는 현재 위치에서 $A_1 + A_2 + \cdots A_N$ km 떨어진 보물이 숨겨진 곳으로 가려고 한다. 걷기 귀찮은 혜아는, 신발의 **추진력(!)**을 사용해서 점프를 하려고 한다. 

혜아가 점프를 할 때, $i$번째 신발을 사용하면, 현재 위치에서 $A_i$ km 떨어진 곳으로 점프할 수 있다. 점프를 하면, 신발은 닳아 없어진다. 모든 신발을 보물이 있는 방향으로 한 번씩 사용하면, 혜아는 보물이 있는 곳에 도달하여 건물주가 될 수 있을 것이다.

혜아가 건물주가 된다는 말을 들은 카흐는, 건물에만 틀어박힐 혜아를 걱정해서, $M$개의 이불을 놓았다. 이불은, 시작 좌표로 부터 보물이 숨겨진 곳을 일직선으로 있는 직선 위에 놓여 있으며, 점프를 해서 착지한 위치가 이불 위라면, 혜아는 이불에서 나오지 않아서 보물을 찾지 못할 것이다. 그는 보물보다, 이불이 더 유혹적이기 때문에, 보물이 있는 곳에 이불이 있으면, 보물을 찾지 않고 바로 이불에 누워버릴 것이다.

혜아를 위해, 이불이 있는 곳을 피해서 보물이 있는 곳에 도착하기 위한 방법을 계산해 주자.

## 입력

첫째 줄에는, $N$과 $M$이 공백으로 구분되어 들어온다. $(1\le M < N \le 100,000)$

둘째 줄에는, $N$개의 수가 공백으로 구분되어 들어온다. 이 중 $i$번째 수는 $A_i$를 의미한다. 추진력은 $1$ 이상 $10^9$ 이하의 서로 다른 수이다.

셋째 줄에는, $M$개의 수가 공백으로 구분되어 들어온다. 이는 이불이 위치한 곳을 의미한다. 수가 $x$라는 것은, 이불이 현재 위치로 부터 보물이 숨겨진 방향으로 $x$ km 떨어져 있다는 것을 의미한다. 이불이 숨겨진 위치는 $1$ 이상 $10^{14}$ 이하의 서로 다른 수이다.

## 출력

보물이 있는 곳에 도착할 수 없으면, 첫째 줄에 -1을 출력한다.

보물이 있는 곳에 도착할 수 있으면, 첫째 줄에 $N$개의 수를 공백으로 구분하여 출력한다. $i$번째 수가 $j$라는 것은, $i$번째 점프를 신발$j$ 를 이용한다는 것을 의미한다. 답이 여러개 인 경우, 아무거나 출력하여도 좋다.

In [ ]:
f = open("input.in")
input = f.readline

In [ ]:
N, M = map(int,input().split())
A = [0] + list(map(int,input().split()))
Trap = list(map(int,input().split()))

def BinarySearch(arr:list, val:int, lo:int=0, hi:int=None):
    if hi is None:
        hi = len(arr)
    while lo<hi:
        mid = (lo+hi)//2
        if arr[mid] < val:
            lo = mid+1
        else:
            hi = mid
    return lo

def updatebit(BITTree , n , i ,v): 
    i += 1
    while i <= n: 
        BITTree[i] += v 
        i += i & (-i) 
        
def getsum(BIT,i): 
    s = 0
    i = i+1
    while i > 0:
        s += BIT[i] 
        i -= i & (-i) 
    return s 

def get_range_sum(BIT, lo, hi):
    return getsum(BIT,hi) - getsum(BIT,lo)

BIT = [0] * (N+1)

def TooManyTraps(M:int, Trap:list, cnt=0, Curr=0, n:int=None, used:list=None) -> list:
    # Priliminary
    if used is None:
        used = [0] * (N+1)
    if n is None:
        n = N
        

    # update Last_idx
    while used[n]:
        n-=1
        
    # case -1.
    if n == N and sum(A) == Trap[-1]:
        return -1
    
    # case 0. 이불이 없는 경우
    if M == 0:
        res = [i for i in range(1,N+1) if not used[i]]
        for c in range(cnt, N):
            updatebit(BIT, N, c, A[res[c-cnt]])
        return res
    
    Case_A, Case_B = False, False
    for i in range(M):
        if Trap[i] > Curr+A[n]:
            break
        elif Trap[i] in [Curr+A[i] for i in range(1,n) if not used[i]]:
            Case_B = True
        elif Trap[i] == Curr+A[n]:
            Case_A = True
            break
    
    if Case_A and Case_B:
        for i in range(1,n):
            if not used[i] and Curr+A[i] not in Trap and Curr+ A[i]+A[n] not in Trap:
                break
        k = BinarySearch(Trap, Curr+A[i]+A[n])
        used[i], used[n] = 1, 1
        tmp = TooManyTraps(M-k, Trap[k:], cnt+2, Curr+A[i]+A[N], n, used)
        updatebit(BIT, N, cnt, A[i])
        updatebit(BIT, N, cnt+1, A[n])
        return [i, n] + tmp
        
    elif Case_A and not Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        used[n] = 1
        tmp = TooManyTraps(M-k-1, Trap[k+1:], cnt+1, Curr+A[N], n, used)
        updatebit(BIT, N, cnt, A[tmp[0]])
        updatebit(BIT, N, cnt+1, A[i])
        return [tmp[0]] + [n] + tmp[1:]
     
    elif not Case_A and Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        used[n] = 1
        tmp = TooManyTraps(M-k, Trap[k:], cnt+1, Curr+A[N], n, used)
        updatebit(BIT, N, cnt, A[n])
        return [n] + tmp
        
    elif not Case_A and not Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        m = Trap[k]
        used[n] = 1
        tmp = [n]+TooManyTraps(M-k-1, Trap[k+1:], cnt+1, Curr+A[N], n, used)
        updatebit(BIT, N, cnt, A[n])
        
        
        t = 0
        for j in range(cnt+1,N):
            if get_range_sum(BIT, cnt, j) == m:
                t = j
                break
        
        if t != 0:
            updatebit(BIT, N, cnt, -A[n]);updatebit(BIT, N, cnt+t+1, -A[tmp[t+1]])
            updatebit(BIT, N, cnt, A[tmp[t+1]]);updatebit(BIT, N, cnt+t+1, A[n])
            tmp[0], tmp[t+1] = tmp[t+1], tmp[0]
        return tmp

res = TooManyTraps(M,Trap)
if res == -1:
    print(-1)
else:
    print(*res)

5 4 1 2 3


In [ ]:
def TooManyTraps(N:int, A:list, M:int, Trap:list, Curr:int=0, n:int=None, used:list=None) -> list:
    # Priliminary
    if used is None:
        used = [0] * (N+1)
    if n is None:
        n = N
    
    # update Last_idx
    while used[n]:
        n-=1
        
    # the only case returning -1
    if n == N and sum(A) == Trap[-1]:
        return -1
    
    # case 0. 이불이 없는 경우
    if M == 0:
        return [i for i in range(1,N+1) if not used[i]]
    
    Case_A, Case_B = False, False
    for i in range(M):
        if Trap[i] > Curr+A[n]:
            break
        elif Trap[i] in [Curr+A[i] for i in range(1,n) if not used[i]]:
            Case_B = True
        elif Trap[i] == Curr+A[n]:
            Case_A = True
            break
    
    if Case_A and Case_B:
        for i in range(1,N):
            if not used[i] and Curr+A[i] not in Trap and Curr + A[i]+A[n] not in Trap:
                break
        k = BinarySearch(Trap, Curr+A[i]+A[n])
        used[i], used[n] = 1, 1
        tmp = TooManyTraps(N, A, M-k, Trap[k:], Curr+A[i]+A[N], n, used)
        return [i, n] + tmp
        
    elif Case_A and not Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        used[n] = 1
        tmp = TooManyTraps(N, A, M-k-1, Trap[k+1:], Curr+A[N], n, used)
        return [tmp[0]] + [n] + tmp[1:]
     
    elif not Case_A and Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        used[n] = 1
        tmp = TooManyTraps(N, A, M-k, Trap[k:], Curr+A[N], n, used)
        return [n] + tmp
        
    elif not Case_A and not Case_B:
        k = BinarySearch(Trap, Curr+A[n])
        m = Trap[k]
        used[n] = 1
        arr = [0,N]+TooManyTraps(N, A, M-k-1, Trap[k+1:], Curr+A[N], n, used)
        
        bit = [0 for i in range(n+1)]
        for index in range(1, n+1):
            update(index, arr[index], arr, bit)
        
        lo,hi=0,n
        while lo<hi:
            mid = (lo + hi) // 2
            if get_range_sum(lo,hi,bit) < m:
                lo = mid+1
            else:
                hi = mid
        t = lo
        
        arr[1], arr[t+1] = arr[t+1], arr[1]

        return arr[1:]

In [ ]:
def BinarySearch(arr:list, val:int, lo:int=0, hi:int=None):
    if hi is None:
        hi = len(arr)
    while lo<hi:
        mid = (lo+hi)//2
        if arr[mid] < val:
            lo = mid+1
        else:
            hi = mid
    return lo

def updatebit(BITTree , n , i ,v): 
    i += 1
    while i <= n: 
        BITTree[i] += v 
        i += i & (-i) 
        
def getsum(BIT,i): 
    s = 0
    i = i+1
    while i > 0:
        s += BIT[i] 
        i -= i & (-i) 
    return s 

def get_range_sum(BIT, lo, hi):
    return getsum(BIT,hi) - getsum(BIT,lo)


In [ ]:
f = open("input.in")
input = f.readline

N, M = map(int,input().split())

# 신발
jump = list(map(int,input().split()))
jump.sort()

sum_jump = sum(jump)

# 이불 위치
trap = [0] * (sum_jump+1)
for t in list(map(int,input().split())):
    trap[t] = 1
print(trap)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
def findTreasure(jump, trap):
    if sum(trap) == 0:
        return jump
    
    case_A = (trap[jump[-1]] == 1) # jump[-1] 에 trap이 위치할 때
    
    case_B = False # jump[0], ... jump[-2] 사이에 trap이 위치할 때
    for j in jump[:-1]:
            if trap[j] == 1:
                case_B = True
                break
    
    order = findTreasure(jump[:-1], [0]+trap[jump[-1]+1:])
    
    if case_A:
        if case_B:
            return '나주엥'
        else:
            return [order[0]] + [jump[-1]] + order[1:]
    else:
        if case_B:
            return [jump[-1]] + order
        else:
            return "나중에 2"

findTreasure(jump, trap)

[9, 4, 5]